In [1]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import warnings
import time 
import string
# import openpyxl
import os

warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
pd.reset_option('display.max_row', None)

In [10]:
df_sleman = pd.read_csv('sleman.csv')
df_sleman

,nama_kecamatan,nama_kelurahan,luas_wilayah_(km2),kepadatan_(jiwa/km2)
0,GAMPING,BALECATUR,9.86,2003.45
1,GAMPING,AMBARKETAWANG,6.28,3145.54
2,GAMPING,BANYURADEN,4.25,4648.00
3,GAMPING,NOGOTIRTO,3.49,5660.17
4,GAMPING,TRIHANGGO,5.62,3514.95
...,...,...,...,...
81,CANGKRINGAN,ARGOMULYO,8.47,2332.23
82,CANGKRINGAN,WUKIRSARI,14.56,1356.73
83,CANGKRINGAN,GLAGAHARJO,7.95,2484.78
84,CANGKRINGAN,KEPUHARJO,8.75,2257.60


# Starbucks

In [4]:
import re
def search_word(text, word, type):
    if (type == "address"): 
        words = text.split(sep=', ')  # Split the text into individual words
    elif (type == "name"):
        words = re.split(r"[', ']", text)
    
    count = 0
    value = False
    # print(words)
    
    for w in words:
        if w == word:
            count+=1
            value = True  # Word found
    return value, count # Word not found

In [ ]:
maps_df = pd.DataFrame()

In [ ]:
def scratch_data (maps_df, jenis, kelurahan, kecamatan): 
    driver.switch_to.default_content()
    searchbox=driver.find_element(By.ID, ('searchboxinput'))

    if (kelurahan == kecamatan): 
        location= f"kelurahan {kelurahan} kecamatan {kecamatan}"
    else: 
        location= f"{kelurahan} {kecamatan}"

    searchbox.send_keys(location)
    searchbox.send_keys(Keys.ENTER)
    time.sleep(2)

    try: 
        kelurahan = driver.find_element(By.CSS_SELECTOR, "[class*=fontHeadlineLarge]").text
        time.sleep(2)
    except NoSuchElementException:
        pass

    searchbox.send_keys(f" {jenis}")
    searchbox.send_keys(Keys.ENTER)
    time.sleep(3)

    entries = driver.find_elements(By.XPATH, "//*[@role='article']")

    for entry in entries:
        name= entry.get_attribute("aria-label")

        if (search_word(name.upper(), jenis.upper(), "name")[0]): 
            link = entry.find_element(By.TAG_NAME, "a")
            
            driver.execute_script("arguments[0].scrollIntoView(true);", link)
            link.click()
            time.sleep(3)

            # Switch to the pop-up window
            for window_handle in driver.window_handles:
                if window_handle != driver.current_window_handle:
                    driver.switch_to.window(window_handle)
                    break

            # driver.switch_to.window(link)
            try: 
                address = driver.find_element(By.CSS_SELECTOR, '[data-item-id = "address"]').get_attribute("aria-label")

            except NoSuchElementException:
                continue
            
            if (kelurahan != kecamatan): 
                if (search_word(address, kelurahan, "address")[0]): 
                    # print(name)
                    # print(address)
                    maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama Starbucks': name, 'Alamat' : address}, ignore_index= True)
            else:
                if (search_word(address, kelurahan, "address")[0]):
                    if (search_word(address, kelurahan, "address")[1] > 1): 
                        maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama Starbucks': name, 'Alamat' : address}, ignore_index= True)
        else: continue
    
    searchbox.clear()
    return maps_df


In [ ]:
i = 0

In [ ]:
from selenium.common.exceptions import StaleElementReferenceException
#Loading Selenium Webdriver 
driver= webdriver.Chrome()
wait = WebDriverWait(driver, 5)
 
#Opening Google maps 
driver.get("https://www.google.com/maps")
time.sleep(3)

for index, row in df_sleman.iterrows():
    try: 
        if (index >= i): 
            i = index
            maps_df = scratch_data(maps_df, "Starbucks", row["nama_kelurahan"], row["nama_kecamatan"])
    except StaleElementReferenceException:
        if (index >= i): 
            i = index
            maps_df = scratch_data(maps_df, "Starbucks", row["nama_kelurahan"], row["nama_kecamatan"])
    

In [ ]:
maps_df

,Kelurahan,Kecamatan,Nama Starbucks,Alamat
0,Sinduadi,MLATI,Starbucks Coffee Jogja City Mall,"Alamat: Jogja City Mall GF #01-02, Jl. Magelan..."
1,Caturtunggal,DEPOK,Starbucks,"Alamat: Plaza Ambarrukmo, Jl. Laksda Adisucipt..."
2,Caturtunggal,DEPOK,Starbucks Uttara Icon,"Alamat: Jl. Kaliurang, Manggung, Caturtunggal,..."
3,Caturtunggal,DEPOK,Starbucks Coffee Seturan - Kledokan,"Alamat: 6CG5+WMR, Kledokan, Caturtunggal, Kec...."
4,Condongcatur,DEPOK,Starbucks Reserve Pakuwon Mall Jogja,"Alamat: 69QX+VF8, Lantai GF Hartono Mall, Jl. ..."
5,Condongcatur,DEPOK,Starbucks Reserve Pakuwon Jogja,"Alamat: Pakuwon mall yogyakarta, Sanggrahan, C..."


In [ ]:
maps_df.to_csv("Starbucks di Sleman.csv", index = False)

# Universitas / Sekolah Tinggi

In [5]:
def search_word_list(text, word_list, type):
    if (type == "address"): 
        words = text.split(sep=', ')  # Split the text into individual words
    elif (type == "name"):
        words = re.split(r"[', ']", text)
    
    count = 0
    value = False
    # print(words)
    
    for word in word_list: 
        for w in words:
            if w == word:
                count+=1
                value = True  # Word found
    return value, count # Word not found

In [ ]:
def scratch_data_list (maps_df, jenis, kelurahan, kecamatan, kategori): 
    driver.switch_to.default_content()
    searchbox=driver.find_element(By.ID, ('searchboxinput'))

    if (kelurahan == kecamatan): 
        location= f"kelurahan {kelurahan} kecamatan {kecamatan}"
    else: 
        location= f"{kelurahan} {kecamatan}"

    searchbox.send_keys(location)
    searchbox.send_keys(Keys.ENTER)
    time.sleep(2)

    try: 
        kelurahan_new = driver.find_element(By.CSS_SELECTOR, "[class*=fontHeadlineLarge]").text
        time.sleep(2)
    except NoSuchElementException:
        pass

    searchbox.send_keys(f" {jenis}")
    searchbox.send_keys(Keys.ENTER)
    time.sleep(3)

    sidebar = driver.find_element(By.XPATH, "//*[@role='feed']")

    # Scroll until the desired text is visible
    i = 0
    while True and i <= 5:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", sidebar)
        i += 1
        
        try:
            end_text_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Anda telah mencapai akhir daftar")]')))
            break  # Break the loop if the text is found
        except:
            pass  # Continue scrolling if the text is not found

    entries = driver.find_elements(By.XPATH, "//*[@role='article']")
    kategori_upper = [kat.upper() for kat in kategori] 

    for entry in entries:
        name= entry.get_attribute("aria-label")

        if (name != kelurahan):

            link = entry.find_element(By.TAG_NAME, "a")
            
            driver.execute_script("arguments[0].scrollIntoView(true);", link)
            link.click()
            time.sleep(3)

            # Switch to the pop-up window
            for window_handle in driver.window_handles:
                if window_handle != driver.current_window_handle:
                    driver.switch_to.window(window_handle)
                    break
            
            try: 
                category = driver.find_element(By.CSS_SELECTOR, "[jsaction = 'pane.rating.category']").text
            except NoSuchElementException:
                continue
            
            if (search_word_list(category.upper(), kategori_upper, "name")[0]): 
                # driver.switch_to.window(link)
                print("Berhasil")
                try: 
                    address = driver.find_element(By.CSS_SELECTOR, '[data-item-id = "address"]').get_attribute("aria-label")

                except NoSuchElementException:
                    continue
                
                if (kelurahan_new != kecamatan): 
                    if (search_word(address, kelurahan_new, "address")[0]): 
                        # print(name)
                        # print(address)
                        maps_df = maps_df.append({'Kelurahan': kelurahan_new, 'Kecamatan': kecamatan, 'Nama Starbucks': name, 'Alamat' : address}, ignore_index= True)
                        print("Data Ditambahkan")
                else:
                    if (search_word(address, kelurahan_new, "address")[0]):
                        if (search_word(address, kelurahan_new, "address")[1] > 1): 
                            maps_df = maps_df.append({'Kelurahan': kelurahan_new, 'Kecamatan': kecamatan, 'Nama Starbucks': name, 'Alamat' : address}, ignore_index= True)
            else: continue
    
    searchbox.clear()
    return maps_df


In [ ]:
univ_df = pd.DataFrame()

In [ ]:
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException  
#Loading Selenium Webdriver 
driver= webdriver.Chrome()
wait = WebDriverWait(driver, 5)
 
#Opening Google maps 
driver.get("https://www.google.com/maps")
time.sleep(3)

category = ["Universitas", "Perguruan"]

for index, row in df_sleman.iterrows():
    try: 
        if (index >= i): 
            i = index
            univ_df = scratch_data_list(univ_df, "Universitas", row["nama_kelurahan"], row["nama_kecamatan"], category)
    except StaleElementReferenceException:
        if (index >= i): 
            i = index
            univ_df = scratch_data_list(univ_df, "Universitas", row["nama_kelurahan"], row["nama_kecamatan"], category)
    except ElementNotInteractableException:
        pass
    except NoSuchElementException: 
        continue

i = 0

Berhasil
Data Ditambahkan
Berhasil
Berhasil
Data Ditambahkan
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Data Ditambahkan
Berhasil
Berhasil
Berhasil


In [15]:
univ_df

NameError: name 'univ_df' is not defined

In [ ]:
univ_df.to_csv("Universitas di Sleman.csv", index = False)

# SMA

In [6]:
def scratch_data_list_2 (maps_df, jenis, kelurahan, kecamatan, kategori, ketentuan): 
    driver.switch_to.default_content()
    searchbox=driver.find_element(By.ID, ('searchboxinput'))

    if (kelurahan == kecamatan): 
        location= f"kelurahan {kelurahan} kecamatan {kecamatan}"
    else: 
        location= f"{kelurahan} {kecamatan}"

    searchbox.send_keys(location)
    searchbox.send_keys(Keys.ENTER)
    time.sleep(2)

    try: 
        kelurahan = driver.find_element(By.CSS_SELECTOR, "[class*=fontHeadlineLarge]").text
        time.sleep(2)
    except NoSuchElementException:
        pass

    searchbox.send_keys(f" {jenis}")
    searchbox.send_keys(Keys.ENTER)
    time.sleep(3)

    sidebar = driver.find_element(By.XPATH, "//*[@role='feed']")

    # Scroll until the desired text is visible
    i = 0
    while True and i <= 5:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", sidebar)
        i += 1
        
        try:
            end_text_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Anda telah mencapai akhir daftar")]')))
            break  # Break the loop if the text is found
        except:
            pass  # Continue scrolling if the text is not found

    entries = driver.find_elements(By.XPATH, "//*[@role='article']")
    kategori_upper = [kat.upper() for kat in kategori] 
    ketentuan_upper = [ket.upper() for ket in ketentuan]

    for entry in entries:
        name= entry.get_attribute("aria-label")

        if (name != kelurahan):

            link = entry.find_element(By.TAG_NAME, "a")
            
            driver.execute_script("arguments[0].scrollIntoView(true);", link)
            link.click()
            time.sleep(3)

            # Switch to the pop-up window
            for window_handle in driver.window_handles:
                if window_handle != driver.current_window_handle:
                    driver.switch_to.window(window_handle)
                    break
            
            try: 
                category = driver.find_element(By.CSS_SELECTOR, "[jsaction = 'pane.rating.category']").text
            except NoSuchElementException:
                continue

            if ((search_word_list(category.upper(), kategori_upper, "name")[0]) or search_word_list(name.upper(), ketentuan_upper, "name")[0]): 
                # driver.switch_to.window(link)
                print("Berhasil")
                try: 
                    address = driver.find_element(By.CSS_SELECTOR, '[data-item-id = "address"]').get_attribute("aria-label")

                except NoSuchElementException:
                    continue
                
                if (kelurahan != kecamatan): 
                    if (kelurahan.split(", ")) == 1:
                        if (search_word(address.upper(), kelurahan.upper(), "address")[0]): 
                            maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama SMA': name, 'Alamat' : address}, ignore_index= True)
                            print("Data Ditambahkan")
                    else:
                        if (search_word(address.upper(), kelurahan.split(", ")[0].upper(), "address")[0]): 
                            maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama SMA': name, 'Alamat' : address}, ignore_index= True)
                            print("Data Ditambahkan")
                else:
                    if (search_word(address.upper(), kelurahan.upper(), "address")[0]):
                        if (search_word(address.upper(), kelurahan.upper(), "address")[1] > 1): 
                            maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama SMA': name, 'Alamat' : address}, ignore_index= True)
            else: continue
    
    searchbox.clear()
    return maps_df


In [7]:
sma_df = pd.DataFrame()

In [14]:
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException
#Loading Selenium Webdriver 
driver= webdriver.Chrome()
wait = WebDriverWait(driver, 5)
 
#Opening Google maps 
driver.get("https://www.google.com/maps")
time.sleep(3)

category = ["Sekolah Menengah Atas", "Atas", "Umum", "Kejuruan"]
keterangan = ["SMA", "SMAN", "MA", "MAN", "SMK", "SMKN", "SMU", "SMAK", "ATAS"]

for index, row in df_sleman.iterrows():
    try: 
        if (index >= i): 
            i = index
            sma_df = scratch_data_list_2(sma_df, "Sekolah Menengah Atas", row["nama_kelurahan"], row["nama_kecamatan"], category, keterangan)
    except StaleElementReferenceException:
        if (index >= i): 
            i = index
            sma_df = scratch_data_list_2(sma_df, "Sekolah Menengah Atas", row["nama_kelurahan"], row["nama_kecamatan"], category, keterangan)
    except NoSuchElementException: 
        continue
    except ElementNotInteractableException:
        pass
i = 0

Berhasil
Data Ditambahkan
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Data Ditambahkan
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Berhasil
Dat

In [16]:
sma_df

,Kelurahan,Kecamatan,Nama SMA,Alamat
0,Tridadi,SLEMAN,SMA & SMK Sulaiman,"Alamat: Jl. Magelang-yogyakarta Km.12, Kel, Wa..."
1,Pandowoharjo,SLEMAN,SMA Negeri 2 Sleman,"Alamat: Brayut, Pandowoharjo, Kec. Sleman, Kab..."
2,Tambakrejo,TEMPEL,Bengkel Jurusan Teknik Logistik SMK Muhammadiy...,"Alamat: 878X+G55, Jl. Tempel-Seyegan, Semampir..."
3,Margorejo,TEMPEL,SMKN 1 Tempel,Alamat: Jl. Magelang - Yogyakarta Jl. Jlegonga...
4,Merdikorejo,TEMPEL,SMK Ma'arif 2 Sleman,"Alamat: 983M+V33, Jl. Bibis, Sono Kulon, Merdi..."
5,Bangun Kerto,TURI,SMK Muhammadiyah 2 Turi,"Alamat: 5, Jl. Agrowisata Jl. Tempel - Turi No..."
6,Donokerto,TURI,SMA Negeri 1 Turi,"Alamat: Jl. Turi Tempel KM. 1 RT. 03 / RW. 23,..."
7,Wono Kerto,TURI,SMK Muhammadiyah 1 Turi,"Alamat: 89X9+FPV, Keringan,, Wono Kerto, Turi,..."
8,Purwobinangun,PAKEM,SMA Darul Hikmah,"Alamat: Jl. Palagan Tentara Pelajar No.Km 15, ..."
9,Harjobinangun,PAKEM,MA Negeri 4 Sleman,"Alamat: 8CM6+HWQ, Dero Wetan, Harjobinangun, K..."


In [78]:
sma_df.to_csv("SMA di Sleman.csv", index = False)

In [17]:
# kalo misalnya ini nge-run tambahan (jadi csvnya udah ada)
sma_df.to_csv("SMA di Sleman.csv", mode = 'a', header = False, index = False)

# Tempat Perbelanjaan

In [19]:
def scratch_data_list_shop (maps_df, jenis, kelurahan, kecamatan, kategori): 
    driver.switch_to.default_content()
    searchbox=driver.find_element(By.ID, ('searchboxinput'))

    if (kelurahan == kecamatan): 
        location= f"kelurahan {kelurahan} kecamatan {kecamatan}"
    else: 
        location= f"{kelurahan} {kecamatan}"

    searchbox.send_keys(location)
    searchbox.send_keys(Keys.ENTER)
    time.sleep(2)

    try: 
        kelurahan = driver.find_element(By.CSS_SELECTOR, "[class*=fontHeadlineLarge]").text
        time.sleep(2)
    except NoSuchElementException:
        pass

    searchbox.send_keys(f" {jenis}")
    searchbox.send_keys(Keys.ENTER)
    time.sleep(3)

    sidebar = driver.find_element(By.XPATH, "//*[@role='feed']")

    # Scroll until the desired text is visible
    i = 0
    while True and i <= 7:
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", sidebar)
        i += 1
        
        try:
            end_text_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[contains(text(), "Anda telah mencapai akhir daftar")]')))
            break  # Break the loop if the text is found
        except:
            pass  # Continue scrolling if the text is not found
    
    entries = driver.find_elements(By.XPATH, "//*[@role='article']")
    kategori_upper = [kat.upper() for kat in kategori] 
    
    # Continue with other operations on the page if needed
    for entry in entries:
        name= entry.get_attribute("aria-label")

        if (name != kelurahan):

            link = entry.find_element(By.TAG_NAME, "a")
            
            driver.execute_script("arguments[0].scrollIntoView(true);", link)
            link.click()
            time.sleep(3)

            # Switch to the pop-up window
            for window_handle in driver.window_handles:
                if window_handle != driver.current_window_handle:
                    driver.switch_to.window(window_handle)
                    break
            
            try: 
                category = driver.find_element(By.CSS_SELECTOR, "[jsaction = 'pane.rating.category']").text
            except NoSuchElementException:
                continue

            if (search_word_list(category.upper(), kategori_upper, "name")[0] and (search_word(name, "Bintang", "name")[0] == False)): #or search_word_list(name.upper(), ketentuan_upper, "name")[0]): 
                # driver.switch_to.window(link)
                print("Berhasil")
                if (name != "88Bintang Perdana II"): 
                    try: 
                        rating = entry.find_element(By.CSS_SELECTOR, "[aria-label*='Bintang ']").text
                        print(rating)
                    except NoSuchElementException:
                        rating = "0,0"
                        continue
                    
                    if (float(rating[:2].replace(",", ".")) > 3.0): 
                        try: 
                            address = driver.find_element(By.CSS_SELECTOR, '[data-item-id = "address"]').get_attribute("aria-label")

                        except NoSuchElementException:
                            continue
                        
                        if (kelurahan != kecamatan): 
                            if (kelurahan.split(", ")) == 1:
                                if (search_word(address.upper(), kelurahan.upper(), "address")[0]): 
                                    maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama': name, 'Alamat' : address}, ignore_index= True)
                                    print("Data Ditambahkan")
                            else:
                                if (search_word(address.upper(), kelurahan.split(", ")[0].upper(), "address")[0]): 
                                    maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama': name, 'Alamat' : address}, ignore_index= True)
                                    print("Data Ditambahkan")
                        else:
                            if (search_word(address.upper(), kelurahan.upper(), "address")[0]):
                                if (search_word(address.upper(), kelurahan.upper(), "address")[1] > 1): 
                                    maps_df = maps_df.append({'Kelurahan': kelurahan, 'Kecamatan': kecamatan, 'Nama': name, 'Alamat' : address}, ignore_index= True)
                                    print("Data Ditambahkan")
                    rating = "0,0"
            else: continue
    
    searchbox.clear()
    return maps_df


In [20]:
i = 0
toko_df = pd.DataFrame()

In [24]:
i = 8

8


In [21]:
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException
#Loading Selenium Webdriver 
driver= webdriver.Chrome()
wait = WebDriverWait(driver, 5)
 
#Opening Google maps 
driver.get("https://www.google.com/maps")
time.sleep(3)

category = ["Pusat Perbelanjaan", "Perbelanjaan", "Toko"]

for index, row in df_sleman.iterrows():
    try: 
        if (index >= i): 
            i = index
            toko_df = scratch_data_list_shop(toko_df, "Pusat Perbelanjaan", row["nama_kelurahan"], row["nama_kecamatan"], category)
    except StaleElementReferenceException:
        if (index >= i): 
            i = index
            toko_df = scratch_data_list_shop(toko_df, "Pusat Perbelanjaan", row["nama_kelurahan"], row["nama_kecamatan"], category)
    except NoSuchElementException: 
        continue
    except ElementNotInteractableException:
        if (index >= i): 
            i = index
            toko_df = scratch_data_list_shop(toko_df, "Pusat Perbelanjaan", row["nama_kelurahan"], row["nama_kecamatan"], category)
    # finally:
    #     searchbox=driver.find_element(By.ID, ('searchboxinput'))
    #     searchbox.clear() 
    #     continue

i = 0

Berhasil
4,8(8)
Data Ditambahkan
Berhasil
Berhasil
Berhasil
5,0(8)
Data Ditambahkan
Berhasil
5,0(2)
Data Ditambahkan
Berhasil
5,0(1)
Data Ditambahkan
Berhasil
Berhasil
Berhasil
5,0(4)
Data Ditambahkan
Berhasil
5,0(1)
Data Ditambahkan
Berhasil
4,4(13)
Data Ditambahkan
Berhasil
Berhasil
Berhasil
4,5(12)
Data Ditambahkan
Berhasil
Berhasil
4,3(4)
Data Ditambahkan
Berhasil
Berhasil
Berhasil
4,3(3)
Data Ditambahkan
Berhasil
4,7(3)
Data Ditambahkan
Berhasil
4,7(3)
Data Ditambahkan
Berhasil
5,0(2)
Data Ditambahkan
Berhasil
4,7(18)
Data Ditambahkan
Berhasil
4,3(3)
Data Ditambahkan
Berhasil
4,9(24)
Data Ditambahkan
Berhasil
Berhasil
5,0(1)
Data Ditambahkan
Berhasil
4,8(4)
Data Ditambahkan
Berhasil
5,0(1)
Data Ditambahkan
Berhasil
Berhasil
4,8(5)
Data Ditambahkan
Berhasil
3,7(3)
Berhasil
4,4(18)
Berhasil
4,0(4)
Data Ditambahkan
Berhasil
5,0(1)
Berhasil
4,4(9)
Berhasil
4,8(21)
Data Ditambahkan
Berhasil
Berhasil
5,0(22)
Data Ditambahkan
Berhasil
5,0(2)
Berhasil
5,0(1)
Berhasil
3,0(2)
Berhasil
5,0(4

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DEA813+48355]
	(No symbol) [0x00D7C4B1]
	(No symbol) [0x00C85358]
	(No symbol) [0x00C6D293]
	(No symbol) [0x00CCE37B]
	(No symbol) [0x00CDC473]
	(No symbol) [0x00CCA536]
	(No symbol) [0x00CA82DC]
	(No symbol) [0x00CA93DD]
	GetHandleVerifier [0x0104AABD+2539405]
	GetHandleVerifier [0x0108A78F+2800735]
	GetHandleVerifier [0x0108456C+2775612]
	GetHandleVerifier [0x00E751E0+616112]
	(No symbol) [0x00D85F8C]
	(No symbol) [0x00D82328]
	(No symbol) [0x00D8240B]
	(No symbol) [0x00D74FF7]
	BaseThreadInitThunk [0x74DE7D59+25]
	RtlInitializeExceptionChain [0x7702B74B+107]
	RtlClearBits [0x7702B6CF+191]


In [22]:
toko_df

,Kelurahan,Kecamatan,Nama,Alamat
0,Balecatur,GAMPING,AGEN NASA BALECATUR,"Alamat: Pasekan Lor 06, RW.04, Nyamplung Kidul..."
1,Balecatur,GAMPING,PEPAK STORE,"Alamat: 57FQ+WX9, Jl. Lembah Wisata, Sembung, ..."
2,Balecatur,GAMPING,Rumah aneka krupuk,"Alamat: 58W2+HCP, Unnamed Road, Pasekan Lor, B..."
3,Balecatur,GAMPING,Warung desi sari,"Alamat: 6835+9W4, Nyamplung Kidul, Balecatur, ..."
4,Balecatur,GAMPING,Ndalem KENTHOES'e JOGJA SNACK,"Alamat: 6854+GX6, Nyamplung Lor, Balecatur, Ke..."
...,...,...,...,...
138,Sidoluhur,GODEAN,Toko Al-Barokah Oleh-oleh Hajji/Umroh,"Alamat: Bulaksawah, Berjo Kulon, Sidoluhur, Ka..."
139,Sidoluhur,GODEAN,Warung sayur dan jajan pasar bu sri,"Alamat: Pirak, Mertosutan rt/rw 001/027, Merto..."
140,Sidoluhur,GODEAN,Omah_Cantik,"Alamat: Tebon xiii rt rw 04 30, Area Sawah, Si..."
141,Sidoluhur,GODEAN,Milagros Godean Iin Intarti,"Alamat: 67QP+HM4, Pandean, Sidoluhur, Kec. God..."


In [23]:
# kalo nge-run dari awal
toko_df.to_csv("Toko di Sleman.csv", index = False)

In [66]:
# kalo nge-run pas kernel udah ganti, alias data di df ufah keulang lagi dari 0, terus kalian mau ngelanjutin di kelurahan selanjutnya, 
# ini kaya append gitu lah 
toko_df.to_csv("Toko di Sleman.csv", mode='a', header=False, index=False)

In [ ]:
# # Set the maximum time to wait for new data to load
# wait = WebDriverWait(driver, 5)

# # Specify a locator for an element that appears after scrolling and new data is loaded
# locator = (By.XPATH, "//*[@role='article']")

# # Function to scroll to the bottom of the page
# def scroll_to_bottom():
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# # Function to check if new data is loaded
# def is_new_data_loaded():
#     try:
#         wait.until(EC.visibility_of_element_located(locator))
#         return True
#     except:
#         return False

# # Scroll and extract data
# data = []

# while True:
#     # Scroll to the bottom of the page
#     scroll_to_bottom()
    
#     # Wait for new data to load
#     if is_new_data_loaded():
#         # Extract the data from the page
#         data_elements = driver.find_elements(By.XPATH, "//*[@role='article']")
#         for element in data_elements:
#             data.append(element.text)
#     else:
#         # No more data is loaded, exit the loop
#         break

In [ ]:
# # Manually scroll through the page to load results
# scroll_pause_time = 1  # Adjust the pause time between scrolls as needed
# scroll_count = 10  # Adjust the number of scrolls to load more results as needed

# for _ in range(scroll_count):
#     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     time.sleep(scroll_pause_time)


In [ ]:
# scroll_container = driver.find_element(By.CSS_SELECTOR, "[attribute_name*='::-webkit-scrollbar']")
# scrollable_height = driver.execute_script("return arguments[0].scrollHeight", scroll_container)
# current_scroll_position = 0
# print(scrollable_height)

# while current_scroll_position < scrollable_height:
#     # Scroll to the bottom of the container
#     driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_container)
#     time.sleep(1)  # Wait for the new results to load
#     current_scroll_position = scrollable_height
#     scrollable_height = driver.execute_script("return arguments[0].scrollHeight", scroll_container)



In [ ]:

    # driver.close()
    # adress= entry.find_element(By.CLASS_NAME, 'section-result-location').text
    # phone = entry.find_element(By.CLASS_NAME, 'section-result-hours-phone-container').text.split(' · ')[-1]
    # print(name)
    # try:
    #     webcontainer= entry.find_element(By.CLASS_NAME, 'section-result-action-container')
    #     website=entry.find_element(By.TAG_NAME, 'a').get_attribute("href")
    # except NoSuchElementException:
    #     website="No website could be found"
    # print (name)
    # print (website)
       
 
    #Try/except  to write the extracted info in the Excel file pass if doessn't exist 
    # try:
    #     maps_df.append(pd.Series([location,name,adress,phone,website]))
    # except IndexError:
    #     pass
 

Starbucks DT Hayam Wuruk
Alamat: Jl. Hayam Wuruk No.8, RT.6/RW.2, Kb. Klp., Kecamatan Gambir, Kota Jakarta Pusat, Daerah Khusus Ibukota Jakarta 10120 
Starbucks
Alamat: RT.1/RW.10, Kramat Pela, Kec. Kby. Baru, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12130 
Starbucks
Alamat: Jl. M.H. Thamrin No.9, RT.2/RW.1, Kb. Sirih, Kec. Menteng, Kota Jakarta Pusat, Daerah Khusus Ibukota Jakarta 10340 
Starbucks
Alamat: Jl. Letjen M.T. Haryono No.9, RT.11/RW.5, Tebet Bar., Jakarta, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12810 
Starbucks Cik Ditiro
Alamat: Jl. Teuku Cik Ditiro No.33, RW.5, Menteng, Kec. Menteng, Kota Jakarta Pusat, Daerah Khusus Ibukota Jakarta 10310 
Starbucks
Alamat: Cyber 2, Lantai GF, Blok X-5 Kav 1-2, Jl. H. R. Rasuna Said, RT.7/RW.2, Kuningan, Kuningan Tim., Kecamatan Setiabudi, Kota Jakarta Selatan, Daerah Khusus Ibukota Jakarta 12950 
Starbucks Plaza Indonesia Lv 4
Alamat: M.H., Jl. K.H. Wahid Hasyim No.15, RT.7/RW.5, Gondangdia, Menteng, Central Jakar